In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
parasharmanas_movie_recommendation_system_path = kagglehub.dataset_download('parasharmanas/movie-recommendation-system')

print('Data source import complete.')


100%|██████████| 165M/165M [00:00<00:00, 246MB/s]

Extracting files...


Data source import complete.


In [ ]:
!pip install scikit-surprise tensorflow wurlitzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357283 sha256=173954cf1ee58fd600049878479fae3c8fd0b80ae6d2d3576703cc3f4828fa6f
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
import scipy.sparse
import tensorflow as tf

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split as sk_train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.preprocessing import normalize
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import os
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from collections import defaultdict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Flatten, Conv1D, MaxPooling1D, ZeroPadding1D
from math import sqrt
import pickle
from google.colab import files
import zipfile
import os
from keras.models import load_model
import joblib

In [ ]:
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d parasharmanas/movie-recommendation-system

# Crie um diretório onde os arquivos extraídos serão armazenados
if not os.path.exists("movie-recommendation-system"):
    os.makedirs("movie-recommendation-system")

with zipfile.ZipFile("movie-recommendation-system.zip", "r") as zip_ref:
    zip_ref.extractall("movie-recommendation-system")


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system
License(s): ODbL-1.0
 98% 162M/165M [00:00<00:00, 303MB/s]
100% 165M/165M [00:00<00:00, 278MB/s]


In [ ]:
movies_file = 'movie-recommendation-system/movies.csv'
movies = pd.read_csv(movies_file)
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
users_file = 'movie-recommendation-system/ratings.csv'
users = pd.read_csv(users_file)
users.head(10)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414


In [ ]:
movie_dataframe = pd.merge(users, movies, on='movieId')

movie_dataframe.head(10)

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
5,8,296,5.0,890489713,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
6,10,296,4.5,1227571308,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
7,12,296,5.0,1119354604,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
8,13,296,5.0,1238029599,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
9,14,296,5.0,1506208897,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


In [ ]:
print('#Total dataset: %d' % len(movie_dataframe))

movie_train_df, movie_test_df = sk_train_test_split(movie_dataframe,
                                   stratify=movie_dataframe['userId'],
                                   test_size=0.2,
                                   random_state=42)
print('#Train set: %d' % len(movie_train_df))
print('#Test set: %d' % len(movie_test_df))

#Total dataset: 25000225
#Train set: 20000180
#Test set: 5000045


In [ ]:
class ColaborativeFilteringRecommender:
    def __init__(self, train_data):
        self.train_data = train_data
        self.algo = None

    def prepare_data(self):
        # Definir o esquema dos dados de entrada
        reader = Reader(rating_scale=(self.train_data['rating'].min(), self.train_data['rating'].max()))
        data = Dataset.load_from_df(self.train_data[['userId', 'movieId', 'rating']], reader)
        return data

    def train(self):
        data = self.prepare_data()
        trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

        # Treinar o algoritmo SVD
        self.algo = SVD()
        self.algo.fit(trainset)

        # Avaliar o modelo
        self.evaluate(testset)

        joblib.dump(self.algo, 'cf_model.joblib')

    def evaluate(self, testset):
        # Avaliar o modelo no conjunto de teste
        predictions = self.algo.test(testset)

        # RMSE e MAE
        rmse = accuracy.rmse(predictions)
        mae = accuracy.mae(predictions)

        # Precision e Recall
        precisions, recalls = self.precision_recall_at_k(predictions, k=5, threshold=3.5)

        # Média de precisão e recall
        mean_precision = sum(prec for prec in precisions.values()) / len(precisions)
        mean_recall = sum(rec for rec in recalls.values()) / len(recalls)

        print(f'EQMR: {rmse}')
        print(f'EMA: {mae}')
        print(f'Precisão: {mean_precision}')
        print(f'Revocação: {mean_recall}')

    def precision_recall_at_k(self, predictions, k=10, threshold=3.5):
        # Função para calcular precisão e recall em k
        user_est_true = defaultdict(list)
        for uid, _, true_r, est, _ in predictions:
            user_est_true[uid].append((est, true_r))

        precisions = {}
        recalls = {}

        for uid, user_ratings in user_est_true.items():
            # Ordenar as previsões do usuário
            user_ratings.sort(key=lambda x: x[0], reverse=True)
            # Considerar as top-k previsões
            n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
            n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
            n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

            precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
            recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        return precisions, recalls

    def predict(self, userId, movieId):
        # Prever a avaliação de um usuário para um filme específico
        return self.algo.predict(userId, movieId).est

    def recommend(self, userId, n=10):
        # Obter recomendações para um usuário
        user_movies = self.train_data[self.train_data['userId'] == userId]['movieId'].tolist()
        all_movies = self.train_data['movieId'].unique()
        predictions = [(movieId, self.predict(userId, movieId)) for movieId in all_movies if movieId not in user_movies]
        predictions.sort(key=lambda x: x[1], reverse=True)
        return predictions[:n]

In [ ]:
cf_recommender = ColaborativeFilteringRecommender(movie_dataframe)
cf_recommender.train()
recommendations = cf_recommender.recommend(userId=1, n=10)
print(recommendations)

RMSE: 0.7777
MAE:  0.5869
EQMR: 0.777675454662334
EMA: 0.5869260887527686
Precisão: 0.8231424852547558
Revocação: 0.42404595409565854
[(159817, 4.49257440314969), (2575, 4.48876218239284), (1247, 4.484543007973137), (36363, 4.464871999307328), (2997, 4.4608890851576986), (89759, 4.446312301805543), (2858, 4.440347960849769), (2959, 4.437267235468439), (127180, 4.430327156045361), (171495, 4.42535291311857)]
